<p><strong><font size="6">WALOUS</font></strong></p>

<p><strong><font size="6">Fusion LC classifications results</font></strong></p>

<p><strong><font size="6">Predict on tiles</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [30]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [68]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile
import subprocess

In [32]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [33]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [34]:
run ../SRC/config.py

In [35]:
print config_parameters

{'outputfolder_classifRF_tiles': '../../../Results/Classif_RF/classif_tiles', 'outputfolder_classifRF': '../../../Results/Classif_RF', 'outputfolder_Logfile': '../../../Results/Log_file', 'permanent_mapset': 'PERMANENT', 'outputfolder_training_sample': '../../../Results/Training_sample', 'locationepsg': '31370', 'outputfolder': '../../../Results', 'list_tiles': '../../../Results/list_tiles', 'pixel_classes_list': '../../../Results/pixel_classes_list', 'gisdb': '../../GRASSDATA', 'location': 'WALOUS_31370', 'outputfolder_classfeatures': '../../../Results/Classification_features', 'PYTHONLIB': '/usr/lib/python2.7', 'njobs': 6, 'rf_trained_model': '../../../Results/Classif_RF/rf_trained_model.rda', 'GISBASE': '/usr/lib/grass76'}


In [36]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [8]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/grass76/bin:/usr/lib/grass76/script:/usr/lib/grass76/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 2159 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/2087,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/2087 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD_LIBRARY_PATH = :/usr/lib/grass76/lib 	
COMPIZ_CONFIG_PROFILE = mate 	
WINDOWPATH = 8 	
GTK_OVERLAY_SCROLLING = 0 	
PYTHONPATH

** GRASS GIS Python libraries **

In [9]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

## Special functions

In [10]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time

In [11]:
## Function that create a R script for the prediction on the current tile
def CreateRScript(tile_cat, r_ncores=2):
    import tempfile
    # Define names, input and output path... 
    classif_features = os.path.join(config_parameters['outputfolder_classfeatures'],"features_tile_%s.csv"%tile_cat).replace("\\", "/")
    classif_result = os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_tile_%s.csv"%tile_cat).replace("\\", "/")
    
    # Create and open new temporary R script
    r_commands = "%s.R"%tempfile.mktemp().replace("\\", "/")
    r_file = open(r_commands, 'w')

    # Create a list with all lines to write in the script
    r_instuction = []

    # Piece of R script for intallation of R package
    install_package = "if(!is.element('%s', installed.packages()[,1])){\n"
    install_package += "cat('\\n\\nInstalling %s package from CRAN\n')\n"
    install_package += "if(!file.exists(Sys.getenv('R_LIBS_USER'))){\n"
    install_package += "dir.create(Sys.getenv('R_LIBS_USER'), recursive=TRUE)\n"
    install_package += ".libPaths(Sys.getenv('R_LIBS_USER'))}\n"
    install_package += "chooseCRANmirror(ind=1)\n"
    install_package += "install.packages('%s', dependencies=TRUE)}"
    r_instuction.append(install_package % ('doParallel', 'doParallel', 'doParallel')) #Package for parallel processing in R
    r_instuction.append(install_package % ('caret', 'caret', 'caret')) #Package for machine learning in R
    r_instuction.append(install_package % ('e1071', 'e1071', 'e1071')) 
    r_instuction.append(install_package % ('randomForest', 'randomForest', 'randomForest')) #Package for Random Forest
    r_instuction.append(install_package % ('pROC', 'pROC', 'pROC')) 

    # Piece of R script for loading libraries and dependencies
    lib_require = "library(%s)\n"
    lib_require += "require(%s)"
    r_instuction.append(lib_require % ('parallel', 'parallel')) 
    r_instuction.append(lib_require % ('doParallel', 'doParallel')) 
    r_instuction.append(lib_require % ('caret', 'caret')) 
    r_instuction.append(lib_require % ('randomForest', 'randomForest')) 
    r_instuction.append(lib_require % ('e1071', 'e1071')) 
    r_instuction.append(lib_require % ('pROC', 'pROC')) 

    # Set number of cores to use
    r_instuction.append("# Set number of cores to use")
    r_instuction.append("usedcores=%s"%r_ncores)
    #r_instuction.append("usedcores=detectCores()-5")
    r_instuction.append("registerDoParallel(usedcores)")

    # Import object statistics as dataframe
    r_instuction.append("# Import object statistics as dataframe")
    r_instuction.append("features <- read.csv('%s', sep=',', header=TRUE, row.names=1)"%classif_features)
    
    # Load trained RF model
    r_instuction.append("# Load trained RF model")
    r_instuction.append("rfModel <- readRDS('%s')"%config_parameters['rf_trained_model'].replace("\\", "/"))
   
    # Predict class 
    r_instuction.append("# Predict class")
    r_instuction.append("predicted <- data.frame(predict(rfModel, features))")
    r_instuction.append("rf_predictions <- data.frame(id=rownames(features), predicted)")
    r_instuction.append("names(rf_predictions) <- c('cat_', 'rf_prediction')")

    # Export to csv
    r_instuction.append("# Export to csv")
    r_instuction.append("write.csv(rf_predictions, file='%s', row.names=FALSE, quote=FALSE)"%classif_result)

    # Write instructions to the R script file
    r_file.write("\n".join(r_instuction))
    r_file.close()
    
    # Return the path to the Rscript file
    return r_commands

In [12]:
def launch_mapset(mapset):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],config_parameters['location'],config_parameters["locationepsg"]))
    # Check if mapset exists
    return_message.append(check_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],config_parameters['location'],mapset))
    # Return
    return return_message

In [13]:
def GetMapsetsAccess():
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Add mapsets with input data to the GRASS GIS research path
        gscript.run_command('g.mapsets', mapset="OBIA", operation="add")
        return_message = "Access to other mapset added"
    except:
        return_message += "ERROR: Add access to other Mapsets failed. Please check for problem."
    return return_message

In [14]:
def DefineComputationRegion(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        return_message = "Working on tile '%s'\n"%tile_cat
        gscript.run_command('g.region', raster='segs_tile_%s'%tile_cat)
        # Print
        return_message += "--> Computational region and MASK defined"
    except:
        return_message += "ERROR: Setting of computional region failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [15]:
def ReclassWithLabel(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        # Create r.reclass rule file
        classif_result = os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_tile_%s.csv"%tile_cat)
        fin = open(classif_result)
        reader = csv.reader(fin)
        reader.next()
        temprulecsv = tempfile.mktemp() # Define the csv output file name
        fout = open(temprulecsv, 'w')
        out = []
        for in_row in reader:
            fout.write("%s=%s"%(in_row[0],in_row[1]))
            fout.write("\n")
        fout.write("*=NULL")
        fout.close()
        ## Reclass segments raster layer to keep only outliers segments, using the reclas_rule.csv file (create temporary raster)
        prediction_raster = 'RfFusion_tile_%s'%tile_cat
        gscript.run_command('g.region', overwrite=True, raster='segs_tile_%s'%tile_cat)
        gscript.run_command('r.reclass', overwrite=True, input='segs_tile_%s'%tile_cat, 
                          output=prediction_raster, rules=temprulecsv)
        os.remove(temprulecsv)
        # Print
        return_message += "--> Segmentation raster reclassed"
    except:
        return_message += "ERROR: Reclassification of segmentation raster failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [40]:
def Colorize(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        gscript.run_command('r.colors', map='RfFusion_tile_%s'%tile_cat, rules=data['color_file'])
        return_message += "--> Colors applied"
    except:
        return_message += "ERROR: Application of colors failed. Please check for problem."%tile_cat
    return return_message

In [41]:
def ExportTiff(tile_cat):
    #Declare empty string that will contain the messages to return
    return_message = ''
    try:
        export_path = os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_tile_%s.tif"%tile_cat)
        gscript.run_command('r.out.gdal', overwrite=True, input='RfFusion_tile_%s'%tile_cat,
                            output=export_path, format='GTiff', createopt='COMPRESS=DEFLATE')
        return_message += "--> Classified raster exported"
    except:
        return_message += "ERROR: Export of classified raster failed for cutline '%s'. Please check for problem."%tile_cat
    return return_message

In [42]:
def GetPrediction(tile_cat):
    import subprocess
    start_tile = start_processing() 
#    print "Start processing on tile %s"%tile_cat
    #Declare empty list for saving output messages
    output_message = [] 

    # Create R script
    r_commands = CreateRScript(tile_cat, r_ncores=4)
    
    # Execute R script
    message = "Running R now. Following output is R output."
    output_message.append(message)
#    print message
    try:
        subprocess.check_call(['Rscript', r_commands], stderr=subprocess.STDOUT, )
    except subprocess.CalledProcessError:
        message =  "There was an error in the execution of the R script.\nPlease check the R output."
        output_message.append(message)
#        print message
    message =  "Finished running R."
    output_message.append(message)
#    print message
    
    # Launch mapset
    message = launch_mapset(tile_cat)  
    [output_message.append(a) for a in message]
#    print "\n".join(message)
    
    # Allow access to other mapset 
    message = GetMapsetsAccess()
    output_message.append(message)
#    print message
    
    # Define computional region and mask
    message = DefineComputationRegion(tile_cat)
    output_message.append(message)
#    print message    
    
    # Reclass 
    message = ReclassWithLabel(tile_cat)
    output_message.append(message)
#    print message
    
    # Colorize 
    message = Colorize(tile_cat)
    output_message.append(message)
#    print message

    # Export 
    message = ExportTiff(tile_cat)
    output_message.append(message)
#    print message
    
    #Print processing time
    message = print_processing_time(start_tile, "Prediction for tile '%s' achieved in "%tile_cat)
    output_message.append(message)
#    print message
    
    #Export Log file
    fout = open(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_%s.txt"%tile_cat),"w")
    [fout.writelines('%s\n'%content) for content in output_message]
    fout.close()

## Import list of tiles from file

In [43]:
import pickle

with open(config_parameters['list_tiles'], 'rb') as outputfile: 
    tile_list = pickle.load(outputfile) 

In [44]:
len(tile_list)

284

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Predict using random forest model (R)

In [45]:
# Set number of cores to use
ncores = 20

In [46]:
# Launch processes in parallel
start_parallel = start_processing()
p = Pool(ncores)
output = p.map(GetPrediction, tile_list[:5])  # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
p.close()
p.join()
# Print
print_processing_time(start_parallel, "Computation (on %s cores) achieved in "%ncores)

'Computation (on 20 cores) achieved in 11.7 seconds'

**Check log file for ERRORS**

In [28]:
# Get list of csv with classification feature of individual tiles
import glob
list_log = glob.glob(os.path.join(config_parameters['outputfolder_Logfile'],"Log_Prediction_tile_*.txt"))
print "%s log files in the folder"%len(list_log)

284 log files in the folder


In [47]:
# Declare new counter
count = 0
# Declare new list that will contain list of tile with error
tile_error_list = []
# Loop on list of log file
for logfile in list_log:
    got_error = False
    tile_num = os.path.splitext(os.path.basename(logfile))[0].split("_")[-1]
    fin = open(logfile, 'r')
    for row in fin:
        if row[:5] == "ERROR":  # If at least one line have error message, the whole file will be counted as 1 error
            got_error = True
    if got_error:    
        count += 1
        tile_error_list.append(tile_num)  # Add tile number to the list
# Print
print "%s tile(s) faced an ERROR during the processing.\n"%count

# Update tile list with only tiles that have ERROR in log 
print "\n".join(["Error on tile %s"%(a) for a in tile_error_list])

0 tile(s) faced an ERROR during the processing.




**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Create VRT with all tiles

## In GRASS

In [49]:
# Work in CLASSIF_VRT
launch_mapset("CLASSIF_VRT")  

["Location 'WALOUS_31370' already exist",
 "'CLASSIF_VRT' mapset created in location 'WALOUS_31370'",
 "You are now working in mapset 'WALOUS_31370/CLASSIF_VRT'"]

In [50]:
# Allow access to other mapset 
gscript.run_command('g.mapsets', mapset=','.join(tile_list), operation="add")    

0

In [54]:
# Get a list of individual classifications
classif_tiled = gscript.list_strings(type='raster', pattern="RfFusion_tile_*")

In [55]:
# Create virtual raster
gscript.run_command('r.buildvrt', overwrite=True, 
                    input=",".join(classif_tiled), 
                    output='RfFusion_walous')

0

In [57]:
# Colorize
gscript.run_command('r.colors', map='RfFusion_walous', rules=data['color_file'])

0

## On the computer

In [73]:
# Path for .vrt
export_path = os.path.join(config_parameters['outputfolder_classifRF'],"RF_fusion_walous.vrt")

In [74]:
# Get a file with list of tiles
tiles_path = glob.glob(os.path.join(config_parameters['outputfolder_classifRF_tiles'],"RF_fusion_tile_*.tif"))
tmp_file = "%s_VRT_LIST.txt"%tempfile.mktemp()
with open(tmp_file, 'w') as f:
    f.writelines("\n".join(tiles_path))
f.close()

In [75]:
# Create VRT
try:
    list_argument = ['gdalbuildvrt', '-resolution', 'average', '-input_file_list', tmp_file, export_path]
    subprocess.check_call(list_argument, stderr=subprocess.STDOUT, )
except subprocess.CalledProcessError:
    print "There was an error in the creation of VRT. Please check."